<a href="https://colab.research.google.com/github/UMB200/pytorch_projects/blob/main/05_pytorch_going_modular_exercise_completed.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 05. PyTorch Going Modular Exercises

Welcome to the 05. PyTorch Going Modular exercise template notebook.

There are several questions in this notebook and it's your goal to answer them by writing Python and PyTorch code.

> **Note:** There may be more than one solution to each of the exercises, don't worry too much about the *exact* right answer. Try to write some code that works first and then improve it if you can.

## Resources and solutions

* These exercises/solutions are based on [section 05. PyTorch Going Modular](https://www.learnpytorch.io/05_pytorch_going_modular/) of the Learn PyTorch for Deep Learning course by Zero to Mastery.

**Solutions:**

Try to complete the code below *before* looking at these.

* See a live [walkthrough of the solutions (errors and all) on YouTube](https://youtu.be/ijgFhMK3pp4).
* See an example [solutions notebook for these exercises on GitHub](https://github.com/mrdbourke/pytorch-deep-learning/blob/main/extras/solutions/05_pytorch_going_modular_exercise_solutions.ipynb).

## 1.0 Turn the code to get the data (from section 1. Get Data) into a Python script, such as `get_data.py`.

* When you run the script using `python get_data.py` it should check if the data already exists and skip downloading if it does.
* If the data download is successful, you should be able to access the `pizza_steak_sushi` images from the `data` directory.

In [48]:
%%writefile get_data.py
"""
Downloads data from source and saves to a target directory.
"""

import os
import zipfile
from pathlib import Path
import requests
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

# Setup path to data folder
data_path = Path("data_path/")
img_path = data_path / "pizza_steak_sushi"

# If the image folder doesn't exists, download it and prepare it
if img_path.is_dir():
  print(f"{img_path} directory already exists")
else:
  print(f"Creating directory {img_path}")
  img_path.mkdir(parents=True, exist_ok=True)
# Download data
url_to_download = "https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi.zip"
file_path = data_path / "pizza_steak_sushi.zip"
with open(file_path, "wb") as f:
  print(f"Downloading data")
  f.write(requests.get(url_to_download).content)
#Unzip data
with zipfile.ZipFile(file_path, "r") as zip_ref:
  print(f"Unzipping: {file_path}")
  zip_ref.extractall(img_path)

# Remove zip file
os.remove(file_path)

Overwriting get_data.py


In [49]:
# Example running of get_data.py
!python get_data.py

data_path/pizza_steak_sushi directory already exists
Unzipping: data_path/pizza_steak_sushi.zip


## 2.0 Use [Python's `argparse` module](https://docs.python.org/3/library/argparse.html) to be able to send the `train.py` custom hyperparameter values for training procedures.
* Add an argument flag for using a different:
  * Training/testing directory
  * Learning rate
  * Batch size
  * Number of epochs to train for
  * Number of hidden units in the TinyVGG model
    * Keep the default values for each of the above arguments as what they already are (as in notebook 05).
* For example, you should be able to run something similar to the following line to train a TinyVGG model with a learning rate of 0.003 and a batch size of 64 for 20 epochs: `python train.py --learning_rate 0.003 batch_size 64 num_epochs 20`.
* **Note:** Since `train.py` leverages the other scripts we created in section 05, such as, `model_builder.py`, `utils.py` and `engine.py`, you'll have to make sure they're available to use too. You can find these in the [`going_modular` folder on the course GitHub](https://github.com/mrdbourke/pytorch-deep-learning/tree/main/going_modular/going_modular).

In [50]:
%%writefile data_setup.py
"""
Contains functionality for creating PyTorch DataLoaders for
image classification data.
"""
import os
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

NUM_WORKERS = os.cpu_count()

def create_dataloaders(
    train_dir: str,
    test_dir: str,
    transform: transforms.Compose,
    batch_size: int,
    num_workers: int=NUM_WORKERS
):
  """Creates training and testing DataLoaders.

  Takes in a training directory and testing directory path and turns
  them into PyTorch Datasets and then into PyTorch DataLoaders.

  Args:
    train_dir: Path to training directory.
    test_dir: Path to testing directory.
    transform: torchvision transforms to perform on training and testing data.
    batch_size: Number of samples per batch in each of the DataLoaders.
    num_workers: An integer for number of workers per DataLoader.

  Returns:
    A tuple of (train_dataloader, test_dataloader, class_names).
    Where class_names is a list of the target classes.
    Example usage:
      train_dataloader, test_dataloader, class_names = \
        = create_dataloaders(train_dir=path/to/train_dir,
                             test_dir=path/to/test_dir,
                             transform=some_transform,
                             batch_size=32,
                             num_workers=4)
  """

  # Use ImageFolder to create dataset(s)
  train_dataset = datasets.ImageFolder(train_dir, transform=transform)
  test_dataset = datasets.ImageFolder(test_dir,transform=transform)

  # Get class names
  class_names = train_dataset.classes

  # Turn datasets to DataLoaders
  train_dataloader = DataLoader(train_dataset,
                                batch_size=batch_size,
                                num_workers=num_workers,
                                shuffle=True,
                                pin_memory=True)
  test_dataloader = DataLoader(dataset=test_dataset,
                               batch_size=batch_size,
                               num_workers=num_workers,
                               shuffle=False,
                               pin_memory=True)

  return train_dataloader, test_dataloader, class_names

Overwriting data_setup.py


In [51]:
%%writefile engine.py

import torch
from tqdm.auto import tqdm
from typing import Dict, List, Tuple
from torch import nn

def train_step_loop(model: torch.nn.Module,
                    data_loader: torch.utils.data.DataLoader,
                    loss_fn: torch.nn.Module,
                    optimizer: torch.optim.Optimizer,
                    device: torch.device) -> Tuple[float, float]:
  """Trains a PyTorch model for a single epoch.

  Turns a target PyTorch model to training mode and then
  runs through all of the required training steps (forward
  pass, loss calculation, optimizer step).

  Args:
    model: A PyTorch model to be trained.
    dataloader: A DataLoader instance for the model to be trained on.
    loss_fn: A PyTorch loss function to minimize.
    optimizer: A PyTorch optimizer to help minimize the loss function.
    device: A target device to compute on (e.g. "cuda" or "cpu").

  Returns:
    A tuple of training loss and training accuracy metrics.
    In the form (train_loss, train_accuracy). For example:

    (0.1112, 0.8743)
  """
  model.train()
  train_loss, train_acc = 0, 0

  for batch, (X, y) in enumerate(data_loader):
    X, y = X.to(device), y.to(device)
    y_pred = model(X)
    loss = loss_fn(y_pred, y)
    train_loss += loss.item()
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    y_pred_class = torch.argmax(torch.softmax(y_pred, dim=1), dim=1)
    train_acc += (y_pred_class == y).sum().item()/len(y_pred)
  train_loss /= len(data_loader)
  train_acc /= len(data_loader)
  return train_loss, train_acc

def test_step_loop(model: torch.nn.Module,
                    data_loader: torch.utils.data.DataLoader,
                   loss_fn: torch.nn.Module,
                   device: torch.device) -> Tuple[float, float]:
  """Tests a PyTorch model for a single epoch.

  Turns a target PyTorch model to "eval" mode and then performs
  a forward pass on a testing dataset.

  Args:
    model: A PyTorch model to be tested.
    dataloader: A DataLoader instance for the model to be tested on.
    loss_fn: A PyTorch loss function to calculate loss on the test data.
    device: A target device to compute on (e.g. "cuda" or "cpu").

  Returns:
    A tuple of testing loss and testing accuracy metrics.
    In the form (test_loss, test_accuracy). For example:

    (0.0223, 0.8985)
  """
  model.eval()
  test_loss, test_acc = 0, 0
  with torch.inference_mode():
    for batch, (X, y) in enumerate(data_loader):
      X, y = X.to(device), y.to(device)
      test_pred_logits = model(X)

      loss = loss_fn(test_pred_logits, y)
      test_loss += loss.item()

      test_pred_labels = test_pred_logits.argmax(dim=1)
      test_acc += (test_pred_labels == y).sum().item()/len(test_pred_labels)
  test_loss /= len(data_loader)
  test_acc /= len(data_loader)
  return test_loss, test_acc

def train(model: torch.nn.Module,
               train_dataloader: torch.utils.data.DataLoader,
               test_dataloader: torch.utils.data.DataLoader,
               optimizer: torch.optim.Optimizer,
               loss_fn: torch.nn.Module,
               epochs: int,
               device: torch.device)-> Dict[str, List]:
  """Trains and tests a PyTorch model.

  Passes a target PyTorch models through train_step() and test_step()
  functions for a number of epochs, training and testing the model
  in the same epoch loop.

  Calculates, prints and stores evaluation metrics throughout.

  Args:
    model: A PyTorch model to be trained and tested.
    train_dataloader: A DataLoader instance for the model to be trained on.
    test_dataloader: A DataLoader instance for the model to be tested on.
    optimizer: A PyTorch optimizer to help minimize the loss function.
    loss_fn: A PyTorch loss function to calculate loss on both datasets.
    epochs: An integer indicating how many epochs to train for.
    device: A target device to compute on (e.g. "cuda" or "cpu").

  Returns:
    A dictionary of training and testing loss as well as training and
    testing accuracy metrics. Each metric has a value in a list for
    each epoch.
    In the form: {train_loss: [...],
                  train_acc: [...],
                  test_loss: [...],
                  test_acc: [...]}
    For example if training for epochs=2:
                 {train_loss: [2.0616, 1.0537],
                  train_acc: [0.3945, 0.3945],
                  test_loss: [1.2641, 1.5706],
                  test_acc: [0.3400, 0.2973]}
  """
  results = {"train_loss": [],
             "train_acc": [],
             "test_loss": [],
             "test_acc": []}
  for epoch in tqdm(range(epochs)):
    train_loss, train_acc = train_step_loop(model=model,
                                            data_loader=train_dataloader,
                                            loss_fn=loss_fn,
                                            optimizer=optimizer,
                                            device=device)
    test_loss, test_acc = test_step_loop(model=model,
                                        data_loader=test_dataloader,
                                        loss_fn=loss_fn,
                                         device=device)
    print(f"Epoch: {epoch+1} | "
          f"train_loss: {train_loss:.4f} | "
          f"train_acc: {train_acc:.4f} | "
          f"test_loss: {test_loss:.4f} | "
          f"test_acc: {test_acc:.4f}")
    get_tensor_val = lambda x: x.item() if isinstance(x, torch.Tensor) else x
    results["train_loss"].append(get_tensor_val(train_loss))
    results["train_acc"].append(get_tensor_val(train_acc))
    results["test_loss"].append(get_tensor_val(test_loss))
    results["test_acc"].append(get_tensor_val(test_acc))
  return results

Overwriting engine.py


In [58]:
%%writefile model_builder.py
"""
    Model architecture copying TinyVGG from:
    https://poloclub.github.io/cnn-explainer/
    Contains PyTorch model code to instantiate a TinyVGG model.
"""

import torch
from torch import nn

class Model_Builder_TinyVGG(nn.Module):
  """Creates the TinyVGG architecture.

    Replicates the TinyVGG architecture from the CNN explainer website in PyTorch.
    See the original architecture here: https://poloclub.github.io/cnn-explainer/

    Args:
      input_shape: An integer indicating number of input channels.
      hidden_units: An integer indicating number of hidden units between layers.
      output_shape: An integer indicating number of output units.
  """

  def __init__(self,
               input_shape: int,
               hidden_units: int,
               output_shape: int) -> None:
               super().__init__()
               self.cnn_block_1 = nn.Sequential(
                   nn.Conv2d(in_channels=input_shape,
                             out_channels=hidden_units,
                             kernel_size=3,
                             stride=1,
                             padding=0),
                   nn.ReLU(),
                   nn.Conv2d(in_channels=hidden_units,
                             out_channels=hidden_units,
                             kernel_size=3,
                             stride=1,
                             padding=0),
                   nn.ReLU(),
                   nn.MaxPool2d(kernel_size=2,
                                stride=2)
               )
               self.cnn_block_2 = nn.Sequential(
                   nn.Conv2d(hidden_units, hidden_units, 3, 1, 0),
                   nn.ReLU(),
                   nn.Conv2d(hidden_units, hidden_units, 3, 1, 0),
                   nn.ReLU(),
                   nn.MaxPool2d(2)
               )
               self.classifier = nn.Sequential(
                   nn.Flatten(),
                   nn.Linear(in_features=hidden_units*13*13,
                             out_features=output_shape)
               )
  def forward(self, x: torch.Tensor):
    return self.classifier(self.cnn_block_2(self.cnn_block_1(x)))

Overwriting model_builder.py


In [59]:
%%writefile utils.py

import torch
from pathlib import Path

def save_model(model: torch.nn.Module,
               target_dir: str,
               model_name: str):
  """Saves a PyTorch model to a target directory.

  Args:
    model: A target PyTorch model to save.
    target_dir: A directory for saving the model to.
    model_name: A filename for the saved model. Should include
      either ".pth" or ".pt" as the file extension.

  Example usage:
    save_model(model=model_0,
               target_dir="models",
               model_name="05_going_modular_tingvgg_model.pth")
  """
  # Create target directory
  target_dir_path = Path(target_dir)
  target_dir_path.mkdir(parents=True,
                        exist_ok=True)

  assert model_name.endswith(".pth") or model_name.endswith(".pt"), "model_name should end with '.pt' or '.pth'"
  model_save_path = target_dir_path / model_name

  print(f"[INFO] Saving model to: {model_save_path}")
  torch.save(obj=model.state_dict(),
             f=model_save_path)

Overwriting utils.py


In [54]:
%%writefile train.py
"""
Trains Python model for image classification using device-agnostic code.
"""

import os
import torch
import data_setup, engine, model_builder, utils
from torchvision import transforms
from timeit import default_timer as timer
import argparse

parser = argparse.ArgumentParser(description="Getting hyperparameters")

parser.add_argument("--num_epochs",
                    default=5,
                    type=int,
                    help="number of epochs to train for")
parser.add_argument("--batch_size",
                    default=32,
                    type=int,
                    help="number of samples per batch")
parser.add_argument("--hidden_units",
                    default=10,
                    type=int,
                    help="number of hidden units in hidden layers")
parser.add_argument("--learning_rate",
                    default=0.001,
                    type=float,
                    help="learning rate to use for model")
parser.add_argument("--test_dir",
                    default="data_path/pizza_steak_sushi/test",
                    type=str,
                    help="directory of test data")
parser.add_argument("--train_dir",
                    default="data_path/pizza_steak_sushi/train",
                    type=str,
                    help="directory of train data")

args = parser.parse_args()

# SETUP HYPERPARAMETERS
NUM_EPOCHS = args.num_epochs
BATCH_SIZE = args.batch_size
HIDDEN_UNITS = args.hidden_units
LEARNING_RATE = args.learning_rate


# SETUP DIRECTORIES
train_dir_scripted = "data_path/pizza_steak_sushi/train"
test_dir_scripted =  "data_path/pizza_steak_sushi/test"

# SETUP TARGET DEVICE
device = "cuda" if torch.cuda.is_available() else "cpu"

# CREATE TRANSFORMS
data_transform = transforms.Compose([transforms.Resize(size=(64, 64)),
                                     transforms.ToTensor()])

# CREATE DATALOADERS USING data_setup.py
train_dataloader_scripted, test_dataloader_scripted, class_names_scripted = data_setup.create_dataloaders(
    train_dir = train_dir_scripted,
    test_dir = test_dir_scripted,
    transform = data_transform,
    batch_size = BATCH_SIZE)

# CREATE MODEL USING model_builder.py
model_saved = model_builder.Model_Builder_TinyVGG(input_shape=3,
                                             hidden_units=HIDDEN_UNITS,
                                             output_shape=len(class_names_scripted)).to(device)

# SETUP LOSS_FN AND OPTIMIZER
loss_fn_cross_entropy = torch.nn.CrossEntropyLoss()
optimizer_adam = torch.optim.Adam(model_saved.parameters(), lr=LEARNING_RATE)

# START TRAINING USING engine.py
star_time = timer()
engine.train(model=model_saved,
             train_dataloader = train_dataloader_scripted,
             test_dataloader = test_dataloader_scripted,
             loss_fn = loss_fn_cross_entropy,
             optimizer = optimizer_adam,
             epochs = NUM_EPOCHS,
             device = device)
end_time = timer()
print(f"[INFO] Total training time: {end_time-star_time:.3f} seconds")

# SAVE MODEL USING utils.py
utils.save_model(model = model_saved,
                 target_dir = "models",
                 model_name = "05_going_modular_script_mode_tinyvgg_model.pth")

Overwriting train.py


In [60]:
# Example running of train.py
!python train.py --num_epochs 5 --batch_size 128 --hidden_units 128 --learning_rate 0.0003

  0% 0/5 [00:00<?, ?it/s]Epoch: 1 | train_loss: 1.0983 | train_acc: 0.3187 | test_loss: 1.0910 | test_acc: 0.3333
 20% 1/5 [00:30<02:03, 30.89s/it]Epoch: 2 | train_loss: 1.0860 | train_acc: 0.3768 | test_loss: 1.0878 | test_acc: 0.3467
 40% 2/5 [01:02<01:33, 31.20s/it]Epoch: 3 | train_loss: 1.0631 | train_acc: 0.3935 | test_loss: 1.0459 | test_acc: 0.4667
 60% 3/5 [01:34<01:03, 31.54s/it]Epoch: 4 | train_loss: 1.0084 | train_acc: 0.5790 | test_loss: 1.0107 | test_acc: 0.4667
 80% 4/5 [02:06<00:31, 31.90s/it]Epoch: 5 | train_loss: 0.9597 | train_acc: 0.5445 | test_loss: 1.0513 | test_acc: 0.4133
100% 5/5 [02:38<00:00, 31.76s/it]
[INFO] Total training time: 158.820 seconds
[INFO] Saving model to: models/05_going_modular_script_mode_tinyvgg_model.pth


## 3. Create a Python script to predict (such as `predict.py`) on a target image given a file path with a saved model.

* For example, you should be able to run the command `python predict.py some_image.jpeg` and have a trained PyTorch model predict on the image and return its prediction.
* To see example prediction code, check out the [predicting on a custom image section in notebook 04](https://www.learnpytorch.io/04_pytorch_custom_datasets/#113-putting-custom-image-prediction-together-building-a-function).
* You may also have to write code to load in a trained model.

In [63]:
%%writefile predict.py

import torch
import torchvision
import argparse
import model_builder

# Create a parser
parser = argparse.ArgumentParser()

# Get an image path
parser.add_argument("--image",
                    help="target image filepath to predict on")
# Get model path
parser.add_argument("--model_path",
                    default="models/05_going_modular_script_mode_tinyvgg_model.pth",
                    type=str,
                    help="target model to use for prediction filepath")

args = parser.parse_args()

# Setup class names
class_names = ["pizza", "steak", "sushi"]

# Setup device
device = "cuda" if torch.cuda.is_available() else "cpu"

# image path
IMG_PATH = args.image
print(f"[INFO] Predicting on {IMG_PATH}")

# Loading in the model
def load_model(model_path=args.model_path):
  model = model_builder.Model_Builder_TinyVGG(input_shape=3,
                                             hidden_units=128,
                                             output_shape=3).to(device)
  print(f"[INFO] Loading in model from: {model_path}")
  model.load_state_dict(torch.load(model_path))
  return model

# Load in model and predict image
def predict_image(image_path=IMG_PATH,
                      model_path=args.model_path):
  # Load in image
  image = torchvision.io.read_image(str(IMG_PATH)).type(torch.float32)
  model = load_model(model_path)

  # Process image to be the same as in model
  image = image / 255.
  transform = torchvision.transforms.Resize(size=(64, 64))
  image = transform(image)

  # Predict the image
  model.eval()
  with torch.inference_mode():
    image = image.to(device)
    pred_logits = model(image.unsqueeze(dim=0))
    pred_probs = torch.softmax(pred_logits, dim=1)
    pred_labels = torch.argmax(pred_probs, dim=1)
    pred_labels_class = class_names[pred_labels]
  print(f"[INFO] Pred class: {pred_labels_class}, Pred prob: {pred_probs.max():.3f}")

if __name__ == "__main__":
  predict_image()

Overwriting predict.py


In [64]:
# Example running of predict.py
!python predict.py --image data_path/pizza_steak_sushi/test/sushi/175783.jpg

[INFO] Predicting on data_path/pizza_steak_sushi/test/sushi/175783.jpg
[INFO] Loading in model from: models/05_going_modular_script_mode_tinyvgg_model.pth
[INFO] Pred class: pizza, Pred prob: 0.485
